In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp submission
# all_local

In [ ]:
#export
import enum
import glob
from pathlib import Path
from sqlalchemy import Table, Column, Integer, Enum, JSON, Text
from steroidsornot.database import Base, Session

In [ ]:
#export
class Label(enum.Enum):
    natural = 1
    steroids = 2
    uncertain = 3
    irrelevant = 4
    deleted = 5
    autoremoved = 6
    bad_natural = 7
    bad_steroids = 8

In [ ]:
#export
class Submission(Base):
    __tablename__ = "submission"

    id = Column(Integer, primary_key=True)
    label = Column(Enum(Label))
    data = Column(JSON)
    image_path = Column(Text)

    def __repr__(self):
        return f"Submission(id={self.id!r}, label={self.label!r}, image_path={self.image_path!r}, data={self.data!r})"

In [ ]:
def synchronize_image_paths(session):
    posts = session.query(Submission).all()
    for index, post in enumerate(posts):
        path = post.image_path

        if path:
            path = Path(path)
            new_path = Path(glob.glob(f'/home/cedar/source/steroids-or-not/data/images/**/{path.name}')[0])  
            if new_path != path:
                post.image_path = str(new_path)
                post.label = new_path.parent.name
                session.add(post)
            
        # else don't worry about it, if there's no stored path there's nothing to update
        
    session.commit()